# Assignment problem

The Assignment problem is one of the most difficult challenges in Supply Chain Management. This can be broken down into different sections such as human resource management, task sharing, purchasing, and so on. We want to concentrate on purchasing in this section.

A fruit distribution company is tasked with purchasing production sets from various suppliers. The company needs to purchase ten pieces of fruit (Apples, Oranges, Bananas, Grapes, Strawberries, Blueberries, Raspberries, Blackberries, Mangoes, Pineapples) and has received quotes from five different suppliers (Max_fruit, B&H, Thomas, Croper, and Wallas). The demand of each fruit for the company is as below:
* Apples: 1000 kg
* Oranges: 1000 kg
* Bananas: 800 kg
* Grapes: 500 kg
* Strawberries: 200 kg
* Blueberries: 100 kg
* Raspberries: 100 kg
* Blackberries: 150 kg
* Mangoes: 500 kg
* Pineapples: 600 kg

Each supplier can produce all ten fruits, but at different prices, and each product has a different minimum order quantity. The company wishes to assign each crop to the suppliers in order to keep the total cost as low as possible.

In [247]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp

In [248]:
fruit_price = pd.read_excel("Fruit_price.xlsx", sheet_name='price').set_index('Supplier')

The unit of each cell is € / kg

In [249]:
fruit_price

,Apples,Oranges,Bananas,Grapes,Strawberries,Blueberries,Raspberries,Blackberries,Mangoes,Pineapples
Supplier,,,,,,,,,,
Max_fruit,0.500,0.375,0.300,1.000,1.750,2.000,2.250,2.500,0.750,1.250
B&H,0.625,0.450,0.325,1.125,1.625,2.125,2.375,2.625,0.875,1.375
Thomas,0.550,0.400,0.350,0.875,1.500,1.750,2.000,2.250,0.625,1.000
Croper,0.575,0.425,0.290,1.250,1.875,2.250,2.500,2.750,1.000,1.500
Wallas,0.400,0.475,0.400,1.375,2.000,2.375,2.625,2.875,1.250,1.750


In [250]:
min_order = pd.read_excel("Fruit_price.xlsx", sheet_name='min_order').set_index('Supplier')

The unit of each cell is kg

In [251]:
min_order

,Apples,Oranges,Bananas,Grapes,Strawberries,Blueberries,Raspberries,Blackberries,Mangoes,Pineapples
Supplier,,,,,,,,,,
Max_fruit,500,400,300,200,100,100,100,100,200,200
B&H,400,300,250,150,80,80,80,80,150,150
Thomas,600,500,400,300,200,200,200,200,300,300
Croper,300,250,200,100,50,50,50,50,100,100
Wallas,450,350,300,250,150,150,150,150,250,250


In [252]:
demand = {
    'Apples': 800,
    'Oranges': 700,
    'Bananas': 300,
    'Grapes': 200,
    'Strawberries': 150,
    'Blueberries': 100,
    'Raspberries': 100,
    'Blackberries': 150,
    'Mangoes': 250,
    'Pineapples': 300,
}

In [253]:
price_table = np.array(fruit_price).astype(float) 
min_table = np.array(min_order).astype(float)

possible_oreder = []
for i in range(len(demand.values())):
    arr = min_table.T[i]
    arr1 = price_table.T[i]
    dm = list(demand.values())[i]
    mask = arr > dm
    arr1[mask] = np.nan
    possible_oreder.append(arr1)

possible_oreder = np.vstack(possible_oreder)
min_cost_allowed_fruit = np.nanmin(possible_oreder, axis=1)

In [254]:
for i in range(len(demand)):
    supplier = list(np.where(possible_oreder[i] == min_cost_allowed_fruit[i]))[0][0]
    print(f'Buy *{list(demand.keys())[i]}* from  *{fruit_price.index[supplier]}* --> Payment: {min_cost_allowed_fruit[i] * list(demand.values())[i]}€')

print('-----------------------------------------------------------')
print(f'Total payment: {np.sum(min_cost_allowed_fruit * list(demand.values()))}€')

Buy *Apples* from  *Wallas* --> Payment: 320.0€
Buy *Oranges* from  *Max_fruit* --> Payment: 262.5€
Buy *Bananas* from  *Croper* --> Payment: 87.0€
Buy *Grapes* from  *Max_fruit* --> Payment: 200.0€
Buy *Strawberries* from  *B&H* --> Payment: 243.75€
Buy *Blueberries* from  *Max_fruit* --> Payment: 200.0€
Buy *Raspberries* from  *Max_fruit* --> Payment: 225.0€
Buy *Blackberries* from  *Max_fruit* --> Payment: 375.0€
Buy *Mangoes* from  *Max_fruit* --> Payment: 187.5€
Buy *Pineapples* from  *Thomas* --> Payment: 300.0€
-----------------------------------------------------------
Total payment: 2400.75€
